In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.0-dev20200702
Eager mode:  True
Hub version:  0.9.0.dev
GPU is available


In [4]:
train = pd.read_csv('train_set.csv')
val = pd.read_csv('val_set.csv')
print(train.sample(5))
print()
print(val.sample(5))

review  rating
35124    CP values ​​awesome awesome awesome quality ...       4
52045  Trade dress with the new no two Packed complet...       4
49194                Send it to slowly Forum Of good use       2
92790  Abit slow and other requested color From that ...       2
30560  Fast delivery ,, order of the umpteenth time i...       5

                                                  review  rating
28655  Khimarnya ,,, very beautiful veil like it. Rec...       4
26691                   The size small than my expected        3
23645   Excellent product quality excellent product p...       5
7742                                      Kulng my order       5
9546   Good, not its ink stuck, nyesel also only buy ...       3


In [19]:
train_data = train['review'].to_numpy()
train_label = train['rating'].to_numpy() - 1
val_data = val['review'].to_numpy()
val_label = val['rating'].to_numpy() - 1

print(train_data[:5])
print(train_label[:5])

['Package price already received fast delivery murah😁 Thank kasih🙏'
 'Goods have to be in good condition. Sellsr fast response post just a little slow because I want wks ice pack for holding the cheese until melted any ice packs dah hehehe no luck there j & t here we ask today also deliver. Thanks seller. Overall ok'
 'kapaditasnya passable bgt .. thank God'
 ' Awesome awesome quality of goods shipped speed awesome value CP'
 'Good bye but have not I go to hopefully cut no obstacles yaa thanksH']
[3 3 3 4 3]


In [55]:
model_url = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1"
hub_layer = hub.KerasLayer(model_url, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[-2.44089868e-02,  6.51175808e-03, -1.13548979e-01,
         3.12415361e-02, -3.14148217e-02,  1.43294245e-01,
         9.75192152e-03,  2.42060840e-01,  1.89854264e-01,
         9.59324837e-02, -1.00796156e-01,  8.97027478e-02,
         3.40812318e-02,  1.81093812e-02, -5.15136421e-02,
        -3.26767936e-02, -5.20219468e-03, -5.42108677e-02,
         6.64904118e-02,  1.09245777e-01,  1.77362472e-01,
        -2.20769346e-01,  2.30763536e-02,  1.34367645e-01,
         4.04630378e-02, -6.36085197e-02,  1.22411456e-02,
        -3.95116061e-02, -1.45113274e-01,  1.21495329e-01,
        -4.51343274e-03,  1.34631038e-01, -3.38962972e-02,
         1.02488570e-01,  7.72230327e-02, -4.02463488e-02,
        -1.04966514e-01,  2.04012878e-02, -5.84231466e-02,
         5.26563451e-02,  1.83728151e-02, -8.29305798e-02,
        -2.13288255e-02, -7.77230114e-02, -2.08468512e-02,
         1.73920929e-01, -1.20822422e-01,  2.07360666e-02,
      

In [56]:
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5)
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_10 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 85        
Total params: 124,644,837
Trainable params: 124,644,837
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.03, momentum=0.9),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [58]:
train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_label))
val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_label))

for data, label in train_ds.take(1):
    print(data)
    print(label)

tf.Tensor(b'Package price already received fast delivery murah\xf0\x9f\x98\x81 Thank kasih\xf0\x9f\x99\x8f', shape=(), dtype=string)
tf.Tensor(3, shape=(), dtype=int64)


In [59]:
BATCH_SIZE = 256
train_ds = train_ds.cache().shuffle(10000).batch(BATCH_SIZE)
val_ds = val_ds.cache().batch(BATCH_SIZE)

In [60]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    verbose=1
)

Epoch 1/10
459/459 [==============================] - 5s 10ms/step - loss: 1.2508 - accuracy: 0.4103 - val_loss: 1.1296 - val_accuracy: 0.4580
Epoch 2/10
459/459 [==============================] - 5s 10ms/step - loss: 1.1039 - accuracy: 0.4724 - val_loss: 1.0934 - val_accuracy: 0.4738
Epoch 3/10
459/459 [==============================] - 4s 10ms/step - loss: 1.0747 - accuracy: 0.4836 - val_loss: 1.0796 - val_accuracy: 0.4804
Epoch 4/10
459/459 [==============================] - 4s 10ms/step - loss: 1.0575 - accuracy: 0.4907 - val_loss: 1.0764 - val_accuracy: 0.4807
Epoch 5/10
459/459 [==============================] - 5s 10ms/step - loss: 1.0444 - accuracy: 0.4953 - val_loss: 1.0681 - val_accuracy: 0.4816
Epoch 6/10
459/459 [==============================] - 5s 10ms/step - loss: 1.0336 - accuracy: 0.4983 - val_loss: 1.0693 - val_accuracy: 0.4824
Epoch 7/10
459/459 [==============================] - 5s 10ms/step - loss: 1.0240 - accuracy: 0.5028 - val_loss: 1.0652 - val_accuracy: 0.4841

In [61]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [62]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    verbose=1
)

Epoch 1/10
459/459 [==============================] - 5s 10ms/step - loss: 0.9873 - accuracy: 0.5203 - val_loss: 1.0631 - val_accuracy: 0.4842
Epoch 2/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9843 - accuracy: 0.5212 - val_loss: 1.0633 - val_accuracy: 0.4829
Epoch 3/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9819 - accuracy: 0.5215 - val_loss: 1.0640 - val_accuracy: 0.4854
Epoch 4/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9792 - accuracy: 0.5234 - val_loss: 1.0669 - val_accuracy: 0.4837
Epoch 5/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9766 - accuracy: 0.5253 - val_loss: 1.0668 - val_accuracy: 0.4809
Epoch 6/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9743 - accuracy: 0.5270 - val_loss: 1.0674 - val_accuracy: 0.4808
Epoch 7/10
459/459 [==============================] - 4s 10ms/step - loss: 0.9717 - accuracy: 0.5263 - val_loss: 1.0682 - val_accuracy: 0.4819